# SimSwap
Reference: https://github.com/neuralchen/SimSwap

## Prepare code

In [ ]:
!pip install --upgrade pip

In [ ]:
%cd /workspace
!git clone https://github.com/woctezuma/SimSwap.git
%cd /workspace/SimSwap/
!git checkout upgrade-insightface

In [ ]:
!pip install insightface==0.7.3 onnxruntime moviepy > /dev/null
!pip install imageio==2.34.0 > /dev/null

### (optional) GPEN

In [ ]:
%cd /workspace
!git clone https://github.com/yangxy/GPEN.git

# Remove the example input
%rm -rf /workspace/GPEN/examples
%mkdir -p /workspace/GPEN/examples/imgs

In [ ]:
%pip install ninja

## Prepare models

### SimSwap

In [ ]:
%cp /workspace/data/antelope.zip /workspace/SimSwap/
%cp /workspace/data/checkpoints.zip /workspace/SimSwap/
%cp /workspace/data/arcface_checkpoint.tar /workspace/SimSwap/arcface_model/

In [ ]:
%cd /workspace/SimSwap

In [ ]:
!wget https://github.com/woctezuma/SimSwap-colab/releases/download/antelope/antelope.zip
!wget -P ./arcface_model https://github.com/woctezuma/SimSwap-colab/releases/download/1.0/arcface_checkpoint.tar
!wget https://github.com/neuralchen/SimSwap/releases/download/1.0/checkpoints.zip
!wget -P ./parsing_model/checkpoint https://github.com/neuralchen/SimSwap/releases/download/1.0/79999_iter.pth
!wget https://github.com/neuralchen/SimSwap/releases/download/512_beta/512.zip

In [ ]:
%cd /workspace/SimSwap
!unzip ./checkpoints.zip  -d ./checkpoints

!unzip 512.zip -d ./checkpoints

!unzip antelope.zip -d ./insightface_func/models/

### (optional) GPEN

In [ ]:
%cp /workspace/data/RetinaFace-R50.pth /workspace/GPEN/weights/
%cp /workspace/data/GPEN-512.pth /workspace/GPEN/weights/

In [ ]:
# %cd /workspace/GPEN/weights
# !wget https://public-vigen-video.oss-cn-shanghai.aliyuncs.com/robin/models/RetinaFace-R50.pth
# !wget https://public-vigen-video.oss-cn-shanghai.aliyuncs.com/robin/models/GPEN-512.pth

# %cp RetinaFace-R50.pth /workspace/data/
# %cp GPEN-512.pth /workspace/data/

## Prepare data

### Download

In [ ]:
%cd /workspace

!curl https://i.imgur.com/QYJOzy7.jpeg -o cpc_ackboo.jpg
!curl https://i.imgur.com/l5MGOws.jpeg -o starwars_meme.jpg

In [ ]:
input_fname = '/workspace/cpc_ackboo.jpg'
output_fname = '/workspace/starwars_meme.jpg'

### Convert to JPG

Images should not be too large, hence the (arbitrary) limitation of 1024 length.

In [ ]:
def get_new_size(img_size,
                 max_allowed_length = 1024):

  if any(max_allowed_length < sz for sz in img_size):
    long_length = max(img_size)
    ratio = max_allowed_length / long_length
  else:
    ratio = 1.0

  new_img_size = [
                  int(ratio*sz)
                  for sz in img_size
                  ]

  return tuple(new_img_size)

In [ ]:
import glob
from pathlib import Path
from PIL import Image

allow_resize = False

for fname in glob.glob('/workspace/*.png'):
  jpg_fname = fname.replace('.png', '.jpg')

  try:
    img = Image.open(fname)
  except FileNotFoundError:
    continue

  new_size = get_new_size(img.size, max_allowed_length = 1024)
  if allow_resize:
    print(f'Resizing from {img.size} to {new_size}')
    img.resize(new_size)

  print(f'Saving to {jpg_fname}')
  img.convert('RGB').save(jpg_fname)

  Path(fname).unlink()

In [ ]:
jpg_input = input_fname.replace('.png', '.jpg')
jpg_output = output_fname.replace('.png', '.jpg')

## Run

In [ ]:
%mkdir -p /workspace/temp/

### Single

In [ ]:
%cd /workspace/SimSwap

!python test_wholeimage_swapsingle.py \
 --no_simswaplogo \
 --use_mask \
 --crop_size 512 \
 --isTrain false  --name people \
 --Arc_path arcface_model/arcface_checkpoint.tar \
 --pic_a_path {jpg_input} \
 --pic_b_path {jpg_output} \
 --output_path /workspace/temp/

### Multi

In [ ]:
%cd /workspace/SimSwap

!python test_wholeimage_swapmulti.py \
 --no_simswaplogo \
 --use_mask \
 --crop_size 512 \
 --isTrain false  --name people \
 --Arc_path arcface_model/arcface_checkpoint.tar \
 --pic_a_path {jpg_input} \
 --pic_b_path {jpg_output} \
 --output_path /workspace/temp/

## (optional) Post-process with GPEN

In [ ]:
%cp /workspace/temp/result_whole_swapsingle.jpg /workspace/GPEN/examples/imgs/
%cp /workspace/temp/result_whole_swapmutil.jpg /workspace/GPEN/examples/imgs/

In [ ]:
%cd /workspace/GPEN
!python face_enhancement.py